In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")
import psycopg2 as psy
from psycopg2 import sql
from io import StringIO
import sqlalchemy
from sqlalchemy import create_engine

import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/src")
    
from mtg_helpers import *


In [2]:
con_details = {"dbname" : 'pay_to_play', 
               "user" : os.environ['PGSQL_P_USER'], 
               "password" : os.environ['PGSQL_P_PWD'], 
               "host" : 'localhost'}  

In [3]:
data_path = '../data/MTGJSONcards/'
file_name = 'AllPrices.json'
file = open(data_path+file_name, 'r')
price_data = json.load(file)['data']

In [4]:
print(price_data.keys())

dict_keys(['00010d56-fe38-5e35-8aed-518019aa36a5', '0001e0d0-2dcd-5640-aadc-a84765cf5fc9', '0003caab-9ff5-5d1a-bc06-976dd0457f19', '0003d249-25d9-5223-af1e-1130f09622a7', '0004a4fb-92c6-59b2-bdbe-ceb584a9e401', '00054115-b2b6-5e22-a694-76fc8639eeb2', '0005d268-3fd0-5424-bc6b-573ecd713aa1', '0005f481-f2d4-53fa-ba37-cfcf5a5f87f1', '0006172e-304e-5f7b-ba48-f21b8da92178', '00066e04-5677-5ca4-8bd0-f7f7781c44a1', '000a83dc-9ee4-528d-866d-2ca2063e718c', '000a85b2-d96c-5a74-81de-5d0a3a0357b3', '000b229e-1a7d-51ff-bd62-4f87682b69b6', '000cd141-db23-50ee-9729-74b6818347d2', '000db0e0-047e-59e2-b7aa-265e656ea91c', '000db6b3-18f7-5108-8834-0d1f01d79c60', '000e097b-ff5f-5a34-b4fc-a1c76e899e4f', '000ed184-57ec-5b1e-a900-5d770a945999', '00102364-7540-511e-8178-04dcc5725f16', '0010f07c-a2e4-5ca5-b91a-fbc5f0380d45', '00115a34-5c66-5183-bb89-7f67f032ca1b', '00141b2c-2eff-5228-8fbe-6383db16769d', '00159000-b5fc-5c4e-b65a-935da2d89add', '0016d01a-6d8a-5007-af8b-dddd68b19c15', '001779ab-ab90-512e-9e9f-cd5b

Assumed per website that data model would be that the keys of this file would be uuids for cards.  This appears to 'confirm' this assumption.  Will also confirm that for a given card uuid, there's a nested dictionary where card formats are the keys, and price data (more nesting) are the values.

In [5]:
price_data[list(price_data.keys())[0]]

{'paper': {'cardkingdom': {'buylist': {'foil': {'2020-12-15': 5.0,
     '2020-12-16': 5.0,
     '2020-12-17': 5.0,
     '2020-12-18': 5.0,
     '2020-12-19': 5.0,
     '2020-12-20': 5.0,
     '2021-01-04': 5.0,
     '2021-01-05': 5.0,
     '2021-01-06': 5.0,
     '2021-01-08': 5.0,
     '2021-01-09': 5.0,
     '2021-01-10': 5.0,
     '2021-01-11': 5.0,
     '2021-01-12': 4.5,
     '2021-01-13': 4.5,
     '2021-01-15': 4.5,
     '2021-01-16': 2.0,
     '2021-01-17': 2.0,
     '2021-01-18': 2.0,
     '2021-01-20': 2.0,
     '2021-01-22': 4.4,
     '2021-01-23': 4.4,
     '2021-01-24': 4.4,
     '2021-01-25': 4.4,
     '2021-01-26': 4.4,
     '2021-01-27': 4.4,
     '2021-01-28': 4.4,
     '2021-01-29': 4.4,
     '2021-01-30': 4.4,
     '2021-01-31': 4.4,
     '2021-02-01': 4.4,
     '2021-02-02': 4.4,
     '2021-02-03': 4.4,
     '2021-02-04': 4.4,
     '2021-02-05': 3.0,
     '2021-02-06': 3.0,
     '2021-02-07': 3.0,
     '2021-02-09': 3.0,
     '2021-02-10': 3.0,
     '2021-02-11': 3.

In [6]:
conn = connect(con_details)

Connecting to PostgreSQL database...
Connection successful


In [7]:
query = sql.SQL(
"""SELECT availability, COUNT(uuid)
FROM DETRITUS
GROUP BY availability
ORDER BY COUNT(uuid) DESC""")
formats_in_files = pd.read_sql(query, conn)
print(formats_in_files)

           availability  count
0  [arena, mtgo, paper]   3497
1         [mtgo, paper]    208


In [8]:
query = sql.SQL(
"""SELECT uuid
FROM core
""")
uuids_in_db = pd.read_sql(query, conn)
print(uuids_in_db.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3705 entries, 0 to 3704
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   uuid    3705 non-null   object
dtypes: object(1)
memory usage: 29.1+ KB
None


For pricing will pull from paper for all pricing since it's present for all cards in my DB and because it's a classic format!

In [10]:
card_format = 'paper'
price_source = 'cardkingdom'
price_list = 'retail'
card_type_order = ['normal','foil']

uuids_in_price = price_data.keys()

# was able to test the below and of the 54805 in price data, can select via intersection 3704 out of 3705 in card data
# print(len(uuids))
# print(len(uuids_price_lookup))

uuids_price_lookup = set(uuids_in_price).intersection(set(uuids_in_db['uuid']))
uuid_prices = []
uuid_fails =[]



In [11]:
#using the 'oldest' price available for a given uuid as it will be
#closer to the likely value at time of use...though data seems limited here.
#168 cards existed that failed due to not being available at paper prices through cardkingdom
#can tackle this once moving to a composite price.

for i in uuids_price_lookup:
    card_prices = get_prices(i, card_format, price_source, price_list, card_type_order,price_data)
    if card_prices is None:
        uuid_fails.append(i)
    else:
        date = min(card_prices.keys())
        price = card_prices[date]
        uuid_prices.append((i, price_source, price, date))

print('number of uuids with failed price searches:')
print(len(uuid_fails))

number of uuids with failed price searches:
168


In [12]:
prices_to_load = pd.DataFrame(uuid_prices, columns = ['uuid', 'price_source', 'price', 'price_date'])

In [13]:
prices_to_load.head()

,uuid,price_source,price,price_date
0,573d9cfa-c531-56de-a0f1-bafcac074eef,cardkingdom,0.25,2020-12-15
1,10bad46d-c75f-5116-983b-5d355ff250de,cardkingdom,0.25,2021-01-17
2,672fd934-b146-5bf7-8455-1a19e0500766,cardkingdom,0.25,2020-12-15
3,6ffbe1cb-8d6f-5e48-9153-b2cb2158d586,cardkingdom,0.25,2020-12-15
4,74e8a0c7-6ae0-533f-8aa1-cb80ffd160ee,cardkingdom,0.25,2020-12-15


In [77]:
add_new_card_data(con_details, prices_to_load,'fixed_prices',conn)

Successful updating of fixed_prices


In [14]:
query = sql.SQL("""
    SELECT core.uuid, name, identifiers
    FROM core
    LEFT JOIN fixed_prices ON fixed_prices.uuid = core.uuid
    JOIN identifiers ON identifiers.uuid = core.uuid
    WHERE fixed_prices.price is Null;""")

price_gap = pd.read_sql(query, conn)


In [20]:
id_test = price_gap.loc[price_gap['name'] == 'Bonecrusher Giant // Stomp','identifiers']

In [30]:
uuid_t = price_gap.loc[price_gap['name'] == 'Bonecrusher Giant // Stomp','uuid']
uuid_t

19    9ec1a801-76eb-57ae-a3a2-b8d2d1b06e8a
49    7060ed02-2559-509e-af4b-0078286641b6
Name: uuid, dtype: object

In [28]:
id_test[19]

"{'mcmId': '400964', 'mcmMetaId': '282074', 'mtgArenaId': '70262', 'mtgjsonV4Id': '1d53af3d-125f-530b-a39e-6ce9ff15c9e9', 'mtgoId': '78352', 'multiverseId': '473077', 'scryfallId': '09fd2d9c-1793-4beb-a3fb-7a869f660cd4', 'scryfallIllustrationId': 'c7f49422-def6-4502-bc02-38aee9924371', 'scryfallOracleId': 'd6d72f5f-8f5d-4180-b514-f22ff5482902', 'tcgplayerProductId': '199035'}"

### attempting to tackle the un-updated prices due to issues w/ gaps in the deck data.   Based on sql query, only 28 cards need prices to complete the deck lookups, so manually looked them up on cardkingdom and will now load and push.

In [38]:
data_path = '../data/MTGJSONcards/'
file_name = 'MTG Data Schema - Sheet3.tsv'
file_df = pd.read_csv(data_path+file_name, delimiter = "\t")
file_df['price_source'] = 'cardkingdom'
file_df['price_date'] = '2021-02-25'
file_df.drop('name', axis = 1)
file_df.drop('num', axis=1)

,uuid,name,price,price_source,price_date
0,d5ea3ded-5799-5625-8a3c-bad72e6aeb39,Agadeem's Awakening,20.99,cardkingdom,2021-02-25
1,c277aee9-4263-5fbe-ad24-57f5533f14da,Barkchannel Pathway,4.99,cardkingdom,2021-02-25
2,e78d0b7a-7b47-59b2-a148-c5a130224adb,Beanstalk Giant,0.25,cardkingdom,2021-02-25
3,af16a633-1a07-5752-8bb4-503c859c8e5c,"Birgi, God of Storytelling",2.49,cardkingdom,2021-02-25
4,f21cc3ef-1bd5-55b6-a5dd-11bdf3713a54,Brazen Borrower,11.99,cardkingdom,2021-02-25
5,f31fdd39-6824-5bf5-b6a1-f8333aeae539,Clearwater Pathway,6.49,cardkingdom,2021-02-25
6,ac4ffd71-aa91-5694-8a0d-b0e303ca5385,"Cosima, God of the Voyage",1.99,cardkingdom,2021-02-25
7,dc222dc4-d430-5485-8b44-fd75d8608aa8,Cragcrown Pathway,5.99,cardkingdom,2021-02-25
8,8dbc37a4-d21a-5539-8e45-ef2955a0456d,Darkbore Pathway,6.99,cardkingdom,2021-02-25
9,e7619d66-eed8-5e0d-9b45-26e299de2ac1,"Egon, God of Death",0.99,cardkingdom,2021-02-25


In [45]:
add_new_card_data(con_details, file_df[['uuid', 'price_source', 'price', 'price_date']],'fixed_prices',conn)

Successful updating of fixed_prices
